# Day 5: Hydrothermal Venture
## Part 1

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

    0,9 -> 5,9
    8,0 -> 0,8
    9,4 -> 3,4
    2,2 -> 2,1
    7,0 -> 7,4
    6,4 -> 2,0
    0,9 -> 2,9
    3,4 -> 1,4
    0,0 -> 8,8
    5,5 -> 8,2

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

    .......1..
    ..1....1..
    ..1....1..
    .......1..
    .112111211
    ..........
    ..........
    ..........
    ..........
    222111....

In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

## Part 2

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

    An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
    An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:

    1.1....11.
    .111...2..
    ..2.1.111.
    ...1.2.2..
    .112313211
    ...1.2....
    ..1...1...
    .1.....1..
    1.......1.
    222111....

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?


## Import libs and data

In [1]:
import numpy as np
import io

In [2]:
# read data
with open('day5_input.txt') as f:
    vents = np.loadtxt((x.replace('->',',') for x in f),delimiter=',', dtype=int)
    vents = np.reshape(vents, (-1, 2, 2))

## Solve

In [5]:
# create map from max x,y dimensions
def make_map(vents):
    global map
    x_dim = vents[:, :, 0].max() + 1
    y_dim = vents[:, :, 1].max() + 1
    map = np.zeros((x_dim, y_dim))

# select pairs of coordinates xx, yy from line number
def xx(line_no):
    return vents[line_no,:,0]

def yy(line_no):
    return vents[line_no,:,1]

def xy1(line_no):
    return vents[line_no,0,:]

def xy2(line_no):
    return vents[line_no,1,:]

# test orientation of vent line
def orientation(line_no):
    if np.all(vents[line_no,0,0] == vents[line_no,1,0]): # check if x coordinates are equal
        return "X"
    elif np.all(vents[line_no,0,1] == vents[line_no,1,1]): # check if y coordinates are equal
        return "Y"
    else:
        return "Diagonal"

# mark vent lines on map
def if_Y(line_no):
    # if orientation is Y, add 1 to (x1 -> x2, y)
    x_min = xx(line_no).min()
    x_max = xx(line_no).max()+1
    y = yy(line_no)[0]
    for x in range(x_min, x_max):
        map[x,y] = map[x,y]+1

def if_X(line_no):
    # if orientation is X, add 1 to (x, y1 -> y2)
    y_min = yy(line_no).min()
    y_max = yy(line_no).max()+1
    x = xx(line_no)[0]
    for y in range(y_min, y_max):
        map[x,y] = map[x,y]+1

# Arrange coordinates to determine if diagonal line is ascending or descending
def find_xy_of_min_x(line_no):
    if vents[line_no,0,0] > vents[line_no,1,0]:
        return vents[line_no,1,:]
    elif vents[line_no,0,0] < vents[line_no,1,0]:
        return vents[line_no,0,:]
    else:
        return "error"

def find_xy_of_max_x(line_no):
    if vents[line_no,0,0] < vents[line_no,1,0]:
        return vents[line_no,1,:]
    elif vents[line_no,0,0] > vents[line_no,1,0]:
        return vents[line_no,0,:]
    else:
        return "error"

# Determine if diagonal is ascending or descending
def ascending_or_descending(line_no):
    # fixed
    y1 = find_xy_of_min_x(line_no)[1]
    y2 = find_xy_of_max_x(line_no)[1]
    if y1 < y2:
        return "ascending"
    elif y1 > y2:
        return "descending"

def if_ascending(line_no):
    # +1 to each point from xy1 to xy2 on map in ascending diagonal
    x = find_xy_of_min_x(line_no)[0]
    y = find_xy_of_min_x(line_no)[1]
    y2 = find_xy_of_max_x(line_no)[1]
    while y <= y2:
        map[x,y] = map[x,y]+1
        x = x+1
        y = y+1

def if_descending(line_no):
    # +1 to each point from xy1 to xy2 in descending diagnoal
    x = find_xy_of_min_x(line_no)[0]
    y = find_xy_of_min_x(line_no)[1]
    y2 = find_xy_of_max_x(line_no)[1]
    while y >= y2:
        map[x,y] = map[x,y]+1
        x = x+1
        y = y-1

# Create map
make_map(vents)

# Mark locations of vent lines on map
for line_no in range(len(vents)):
    axis = orientation(line_no)
    if axis == 'Y':
        if_Y(line_no)
    elif axis == 'X':
        if_X(line_no)
    elif axis == 'Diagonal':
        pass
        direction = ascending_or_descending(line_no)
        if direction == 'ascending':
            if_ascending(line_no)
        elif direction == 'descending':
            if_descending(line_no)

# Count values >= 2 on map
np.count_nonzero(map>=2)

21104